# Exercise 2: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. Submission includes this notebook only with the exercise number and your ID as the filename. For example: `hw2_123456789_987654321.ipynb` if you submitted in pairs and `hw2_123456789.ipynb` if you submitted the exercise alone.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
1. Write your functions in this notebook only. **Do not create Python modules and import them**.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. **Do not import anything else.**
1. Your code must run without errors. Make sure your `numpy` version is at least 1.15.4 and that you are using at least python 3.6. Changes of the configuration we provided are at your own risk. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Answers to qualitative questions should be written in **markdown** cells (with $\LaTeX$ support). Answers that will be written in commented code blocks will not be checked.

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Construct a decision tree algorithm.
4. Prune the tree to achieve better results.
5. Visualize your results.

# I have read and understood the instructions: *** ,206328734 ***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Warmup - OOP in python

Our desicion tree will be implemented using a dedicated python class. Python classes are very similar to classes in Java.


You can use the following [site](https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/) to learn about classes in python.

In [ ]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)

In [ ]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
n.children

## Data preprocessing

For the following exercise, we will use a dataset containing mushroom data `agaricus-lepiota.csv`. 

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous
one (=there are only two classes **edible** and **poisonous**). 
    
The dataset contains 8124 observations with 22 features:
1. cap-shape: bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
4. bruises: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g,green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

First, we will read and explore the data using pandas and the `.read_csv` method. Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [2]:
# load dataset
data = pd.read_csv('agaricus-lepiota.csv')

One of the advantages of the Decision Tree algorithm is that almost no preprocessing is required. However, finding missing values is always required.

In [3]:
#############################################################################
# TODO: Find the columns with missing values and remove them from the data.#
#############################################################################
missing_values = ["n/a", "na", "--"]
data = pd.read_csv('agaricus-lepiota.csv',na_values = missing_values)
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################

We will split the dataset to `Training` and `Testing` datasets.

In [4]:
from sklearn.model_selection import train_test_split
# Making sure the last column will hold the labels
X, y = data.drop('class', axis=1), data['class']
X = np.column_stack([X,y])
# split dataset using random_state to get the same split each time
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)

Training dataset shape:  (6093, 22)
Testing dataset shape:  (2031, 22)


In [ ]:
y.shape

## Impurity Measures

Impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Implement the functions `calc_gini` and `calc_entropy`. You are encouraged to test your implementation (10 points).

In [5]:
def calc_gini(data):
    """
    Calculate gini impurity measure of a dataset.
 
    Input:
    - data: any dataset where the last column holds the labels.
    
 
    Returns the gini impurity.    
    """
    gini = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if len(data.shape) == 1:
        label = np.array(data)
    else:
        label = np.array(data[:, -1])
    counts = np.array(np.unique(label, return_counts=True)).T
    counts = np.square((np.array(counts[:, -1])) / len(label))
    gini = np.sum(counts)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return 1 - gini

In [6]:
def calc_entropy(data):
    """
    Calculate the entropy of a dataset.

    Input:
    - data: any dataset where the last column holds the labels.

    Returns the entropy of the dataset.    
    """
    entropy = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if len(data.shape) == 1:
        label = np.array(data)
    else:
        label = np.array(data[:, -1])
    counts = np.array(np.unique(label, return_counts=True)).T
    prob = (np.array(counts[:, -1])) / len(label)
    entropy = - np.sum(prob * np.log2(np.array(prob, dtype=float)))
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return entropy

In [7]:
##### Your Tests Here #####
calc_gini(X),calc_entropy(X)

(0.4995636322379775, 0.9993703627906085)

## Goodness of Split

Given a feature the Goodnees of Split measures the reduction in the impurity if we split the data according to the feature.
$$
\Delta\varphi(S, A) = \varphi(S) - \sum_{v\in Values(A)} \frac{|S_v|}{|S|}\varphi(S_v)
$$

In our implementation the goodness_of_split function will return either the Goodness of Split or the Gain Ratio as learned in class. You'll control the return value with the `gain_ratio` parameter. If this parameter will set to False (the default value) it will return the regular Goodness of Split. If it will set to True it will return the Gain Ratio.
$$
GainRatio(S,A)=\frac{InformationGain(S,A)}{SplitInformation(S,A)}
$$
Where:
$$
InformationGain(S,A)=Goodness\ of\ Split\ calculated\ with\ Entropy\ as\ the\ Impurity\ function \\
SplitInformation(S,A)=- \sum_{a\in A} \frac{|S_a|}{|S|}\log\frac{|S_a|}{|S|}
$$
NOTE: you can add more parameters to the function and you can also add more returning variables (The given parameters and the given returning variable should not be touch). (10 Points)

In [8]:
def goodness_of_split(data, feature, impurity_func,gain_ratio=False):
    """
    Calculate the goodness of split of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.
    - impurity func: a function that calculates the impurity.
    - gain_ratio: goodness of split or gain ratio flag.

    Returns the goodness of split (or the Gain Ration).  
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    goodness = 0
    vals, counts = np.unique(data[:, feature], return_counts=True)
    weighted_gain = 0
    if gain_ratio:
        total_entropy = calc_entropy(data)
        for i in range(len(vals)):
            weighted_gain += (counts[i] / data.shape[0]) * calc_entropy(data[data[:, feature] == vals[i]])
        information_gain = total_entropy - weighted_gain
        div = (- np.sum(counts / data.shape[0] * np.log2(counts / data.shape[0])))
        if not div:
            goodness = 0
        else:
            goodness = information_gain / div
    else:
        total_impurity = impurity_func(data)
        for i in range(len(vals)):
            weighted_gain += (counts[i] / data.shape[0]) * impurity_func(data[data[:, feature] == vals[i]])
        goodness = total_impurity - weighted_gain
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return goodness    

## Building a Decision Tree

Use a Python class to construct the decision tree. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the feature and value used for the split and its children.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes categorical data. In this exercise, when splitting a node create the number of children needed according to the attribute unique values.

Complete the class `DecisionNode`. The structure of this class is entirely up to you. 

Complete the function `build_tree`. This function should get the training dataset and the impurity as inputs, initiate a root for the decision tree and construct the tree according to the procedure you learned in class. (30 points)

In [64]:
class DecisionNode:
    """
    This class will hold everything you require to construct a decision tree.
    The structure of this class is up to you. However, you need to support basic 
    functionality as described above. It is highly recommended that you 
    first read and understand the entire exercise before diving into this class.
    """
    def __init__(self, data, feature, value, parent=None):
        self.feature = feature  # column index of criteria being tested
        self.value = value
        self.children = []
        self.parent = parent
        self.count = None
        self.data = data
        self.depth = 0

    def add_child(self, node):
        self.children.append(node)

    def is_leaf(self):
        return len(self.children) == 0

    def count_nodes(self):
        x = 1
        for j in self.children:
            x += j.count_nodes()
        return x

    @staticmethod
    def attribute_to_class(data, impurity_func, gain_ratio=False):
        max_gain = -1.0
        max_index = 0
        for feature in range(data.shape[1] - 1):
            curr_goodness = goodness_of_split(data, feature, impurity_func, gain_ratio)
            if curr_goodness > max_gain:
                max_gain = curr_goodness
                max_index = feature
        return max_index, max_gain

    def add_tree(self, impurity, gain_ratio=False, chi=1, max_depth=1000,root_tree_depth=0):

        if self.depth == max_depth:
            return

        if not impurity(self.data):
            return
        
        if self.data.shape[1] == 1:
            return

        max_index, max_gain = self.attribute_to_class(self.data, impurity, gain_ratio)
        self.feature = max_index
        self.value = max_gain       

        if max_gain == 0:
            return

        labels = np.unique(self.data[:, max_index])
        for val in labels:
            new_child = DecisionNode(self.data[np.where(self.data[:, max_index] == val)],max_index, val, self)
            new_child.depth = self.depth + 1
            tree_depth = max(root_tree_depth, new_child.depth)
            self.add_child(new_child)
        for child in self.children:
            child.add_tree(impurity, gain_ratio, chi, max_depth,tree_depth)

In [65]:
def build_tree(data, impurity, gain_ratio=False, chi=1, max_depth=1000, parent=None):
    root = DecisionNode(data, -1, -1)
    root_tree_depth = 0
    root.add_tree(impurity, gain_ratio, chi, max_depth,root_tree_depth)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return root

In [66]:
# python supports passing a function as an argument to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini) # gini and goodness of split
tree_entropy = build_tree(data=X_train, impurity=calc_entropy) # entropy and goodness of split
tree_entropy_gain_ratio = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True) # entropy and gain ratio

## Tree evaluation

Complete the functions `predict` and `calc_accuracy`. (10 points)

In [67]:
def predict(node, instance):
    """
    Predict a given instance using the decision tree
 
    Input:
    - root: the root of the decision tree.
    - instance: an row vector from the dataset. Note that the last element 
                of this vector is the label of the instance.
 
    Output: the prediction of the instance.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    prediction = None
    now = node
    back = None
    while len(now.children) != 0 and back != now:
        next_child_to_go = False
        for children in now.children:
            if children.value == instance[now.feature]:
                back = now
                now = children
                next_child_to_go = True
                break
        if not next_child_to_go:
            break
    values, counter = np.unique(now.data[:, -1], return_counts=True)
    prediction = values[np.argmax(counter)]
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return prediction

In [68]:
def calc_accuracy(node, dataset):
    """
    Predict a given dataset using the decision tree
 
    Input:
    - node: a node in the decision tree.
    - dataset: the dataset on which the accuracy is evaluated
 
    Output: the accuracy of the decision tree on the given dataset (%).
    """
    accuracy = 0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    goodpredict = 0
    for fet in dataset:
        curr_predict = predict(node, fet)
        if curr_predict == fet[-1]:
            goodpredict += 1
    accuracy = goodpredict / (dataset.shape[0])
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return accuracy 

After building the three trees using the training set, you should calculate the accuracy on the test set. For each tree print the training and test accuracy. Select the tree that gave you the best test accuracy. For the rest of the exercise, use that tree (when you asked to build another tree use the same impurity function and same gain_ratio flag). 

In [70]:
one = calc_accuracy(tree_gini, X_test)
two = calc_accuracy(tree_entropy, X_test)
three = calc_accuracy(tree_entropy_gain_ratio, X_test)
print(one,two,three)
#best tree is tree_entropy_gain_ratio
tree_gini.count_nodes()

0.5061546036435254 0.5061546036435254 0.5061546036435254


3231

## Post pruning

Iterate over all nodes in the tree that have at least a single child which is a leaf. For each such node, replace it with its most popular class. Calculate the accuracy on the testing dataset, pick the node that results in the highest testing accuracy and permanently change it in the tree. Repeat this process until you are left with a single node in the tree (the root). Finally, create a plot of the training and testing accuracies as a function of the number of nodes in the tree. (15 points)

In [ ]:
def post_pruning(root, train_data, test_data):
    
    acuury = {
        'train': [calc_accuracy(root, train_data)],
        'test': [calc_accuracy(root, test_data)]
    }
    curr_data = {
        'train': train_data,
        'test': test_data
    }

    nodes_size = [root.count_nodes()]
    node = root
    while not root.is_leaf():
        curr_leaves = {}
        prune_leaves(root, node, curr_data, curr_leaves)
        best = max(curr_leaves.keys(), key=lambda l: curr_leaves[l])

        best.children = []
        nodes_size.append(root.count_nodes())
        acuury['test'].append(curr_leaves[best])
        acuury['train'].append(calc_accuracy(root, curr_data['train']))
    return acuury, nodes_size


def prune_leaves(root, node, curr_data, curr_leaves):
    if node.is_leaf():
        children = np.copy(node.parent.children)
        if node.parent is not None:
            node.parent.children = []
        curr_leaves[node.parent] = calc_accuracy(root, curr_data['test'])
        node.parent.children = children
        return

    for child in node.children:
        prune_leaves(root, child, curr_data, curr_leaves)
        
a , b = post_pruning(tree_gini, X_train, X_test) 
c , d = post_pruning(tree_entropy, X_train, X_test)
e , f = post_pruning(tree_entropy_gain_ratio, X_train, X_test)

In [ ]:
print(a,b)
print(c,d)
print(e,f)
arr_a_train = np.array(a['train'][:-1])
arr_a_test = np.array(a['test'][:-1])
arr_b = np.array(b[1:])
plt.plot(arr_b,arr_a_train, label="train")
plt.plot(arr_b,arr_a_test, label="test")
plt.xlabel('number of nodes')
plt.ylabel('Accuracy %')
plt.title('Accuracy as a function of the number of nodes in the tree=tree_gini')
plt.legend()
plt.show()
arr_c_train = np.array(c['train'][:-1])
arr_c_test = np.array(c['test'][:-1])
arr_d = np.array(d[1:])
plt.plot(arr_d,arr_c_train, label="train")
plt.plot(arr_d,arr_c_test, label="test")
plt.xlabel('number of nodes')
plt.ylabel('Accuracy %')
plt.title('Accuracy as a function of the number of nodes in the tree=tree_entropy')
plt.legend()
plt.show()
arr_e_train = np.array(e['train'][:-1])
arr_e_test = np.array(e['test'][:-1])
arr_f = np.array(f[1:])
plt.plot(arr_f,arr_e_train, label="train")
plt.plot(arr_f,arr_e_test, label="test")
plt.xlabel('number of nodes')
plt.ylabel('Accuracy %')
plt.title('Accuracy as a function of the number of nodes in the tree')
plt.legend()
plt.show()

## Chi square pre-pruning

Consider the following p-value cut-off values: [1 (no pruning), 0.5, 0.25, 0.1, 0.05, 0.0001 (max pruning)]. For each value, construct a tree and prune it according to the cut-off value. Next, calculate the training and testing accuracy. On a single plot, draw the training and testing accuracy as a function of the tuple (p-value, tree depth). Mark the best result on the graph with red circle. (15 points)

In [ ]:
### Chi square table values ###
# The first key is the degree of freedom 
# The second key is the p-value cut-off
# The values are the chi-statistic that you need to use in the pruning
chi_table = {1: {0.5 : 0.45,
                 0.25 : 1.32,
                 0.1 : 2.71,
                 0.05 : 3.84,
                 0.0001 : 100000},
             2: {0.5 : 1.39,
                 0.25 : 2.77,
                 0.1 : 4.60,
                 0.05 : 5.99,
                 0.0001 : 100000},
             3: {0.5 : 2.37,
                 0.25 : 4.11,
                 0.1 : 6.25,
                 0.05 : 7.82,
                 0.0001 : 100000},
             4: {0.5 : 3.36,
                 0.25 : 5.38,
                 0.1 : 7.78,
                 0.05 : 9.49,
                 0.0001 : 100000},
             5: {0.5 : 4.35,
                 0.25 : 6.63,
                 0.1 : 9.24,
                 0.05 : 11.07,
                 0.0001 : 100000},
             6: {0.5 : 5.35,
                 0.25 : 7.84,
                 0.1 : 10.64,
                 0.05 : 12.59,
                 0.0001 : 100000},
             7: {0.5 : 6.35,
                 0.25 : 9.04,
                 0.1 : 12.01,
                 0.05 : 14.07,
                 0.0001 : 100000},
             8: {0.5 : 7.34,
                 0.25 : 10.22,
                 0.1 : 13.36,
                 0.05 : 15.51,
                 0.0001 : 100000},
             9: {0.5 : 8.34,
                 0.25 : 11.39,
                 0.1 : 14.68,
                 0.05 : 16.92,
                 0.0001 : 100000},
             10: {0.5 : 9.34,
                  0.25 : 12.55,
                  0.1 : 15.99,
                  0.05 : 18.31,
                  0.0001 : 100000},
             11: {0.5 : 10.34,
                  0.25 : 13.7,
                  0.1 : 17.27,
                  0.05 : 19.68,
                  0.0001 : 100000}}

In [ ]:
#### Your code here ####

Build the best 2 trees:
1. tree_post_pruning - the best tree according to post pruning
1. tree_chi - the best tree according to chi square pruning

In [ ]:
#### Your code here ####

## Number of Nodes

Of the two trees above we will choose the one with fewer nodes. Complete the function counts_nodes and print the number of nodes in each tree. (5 points) 

In [ ]:
def count_nodes(node):
    """
    Count the number of node in a given tree
 
    Input:
    - node: a node in the decision tree.
 
    Output: the number of node in the tree.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################  
    return node.count_nodes()
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

## Print the tree

Complete the function `print_tree` and execute it on your chosen tree. Your tree should be visualized clearly. You can use the following example as a reference:
```
[ROOT, feature=X0],
  [X0=a, feature=X2]
    [X2=c, leaf]: [{1.0: 10}]
    [X2=d, leaf]: [{0.0: 10}]
  [X0=y, feature=X5], 
    [X5=a, leaf]: [{1.0: 5}]
    [X5=s, leaf]: [{0.0: 10}]
  [X0=e, leaf]: [{0.0: 25, 1.0: 50}]
```
In each brackets:
* The first argument is the parent feature with the value that led to current node
* The second argument is the selected feature of the current node
* If the current node is a leaf, you need to print also the labels and their counts

(5 points)

In [ ]:
# you can change the function signeture
def print_tree(node,labels, depth=0, parent_feature='ROOT'):
    '''
    prints the tree according to the example above

    Input:
    - node: a node in the decision tree

    This function has no return value
    '''
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    num_of_spaces = depth * "  "
    if node.value == -1:
        print(f"[ROOT, feature={node.feature}],")
    elif node.is_leaf():
        first = np.count_nonzero(node.data[:, -1] == labels[0])
        second = np.count_nonzero(node.data[:, -1] == labels[1])
        to_str1 = f"{labels[0]}: {first}" if first != 0 else ""
        to_str2 = f"{labels[1]}: {second}" if second != 0 else ""
        final = "" if (first == 0 or second == 0) else ", "
        print(num_of_spaces, f"[X{parent_feature}={node.value}, leaf]: [{to_str1}{final}{to_str2}]")
    else:
        print(num_of_spaces, f"[X{parent_feature}={node.value}, feature=X{node.feature}],")
    for child in node.children:
        print_tree(child, labels, depth=depth + 1, parent_feature=node.feature)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

In [ ]:
# print_tree(tree_gini,('p', 'e'))